# Comments

This is an update of the previous version where we had a CV score of 0.946, it was a blend model using effb6 and effb7.

In this script we will only use effb6, remember that we stratified our tf records using the client id to avoid leakage, that is why previous version have a better cv score.

Out of folds score in this version is 0.951, their were 2 major changes, the first one is to use image size of 768 x 768 and the second one is to use Binary Focal Loss.

Previous ensemble score without client ids stratification:

Public leaderboard: 0.959 
CV Ensemble: 0.9563

You can find the link of the dataset here: 

* https://www.kaggle.com/ragnar123/ranzcr-tf-records-768-stratified. 

Used a tf.data pipeline with tpu for training in kaggle, you can find the link of the script here: 

* https://www.kaggle.com/ragnar123/ranzcr-efficientnetb6-baseline

Finally, this is the inference pipeline where we also use tf.data but we extract each image directly from the directory (not using tf records because of hidden test set)

I hope this help tensorflow users because high lb score are mainly pytorch scripts.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import random
import math
from sklearn import metrics
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from kaggle_datasets import KaggleDatasets
!pip install ../input/cassava-models/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/cassava-models/efficientnet-1.1.0-py3-none-any.whl
import efficientnet.tfkeras as efn
import glob

In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 15
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [768, 768]
# Seed
SEED = 123
# Learning rate
LR = 0.001
# Test time augmentation rounds
TTA = 5
# Verbosity
VERBOSE = 2
# Number of classes
N_CLASSES = 11

# Training filenames directory
TEST_FILENAMES = '../input/ranzcr-clip-catheter-line-classification/test/*.jpg'

In [ ]:
# Data augmentation function
def data_augment(image, StudyInstanceUID):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)
    return image, StudyInstanceUID

# Function to decode our images
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# Function to get StudyInstanceUID
def get_image_name(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    StudyInstanceUID = parts[-1]
    return StudyInstanceUID

# Function to read our image and get our StudyInstanceUID
def read_image(file_path):
    StudyInstanceUID = get_image_name(file_path)
    image = tf.io.read_file(file_path)
    image = decode_image(image)
    return image, StudyInstanceUID

# Function to get our test data were we add a flag for test time augmentation
def get_tta(filenames, tta = False):
    dataset = tf.data.Dataset.list_files(filenames, shuffle = False)
    dataset = dataset.map(read_image, num_parallel_calls = AUTO)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    if tta:
        dataset = dataset.repeat() 
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

NUM_TESTING_IMAGES = len(os.listdir('../input/ranzcr-clip-catheter-line-classification/test/'))

In [ ]:
# Function to create our EfficientNetB6 model
def get_model():
    
    with strategy.scope():
        
        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3))
        x = efn.EfficientNetB6(include_top = False, weights = None)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        output = tf.keras.layers.Dense(N_CLASSES, activation = 'sigmoid')(x)
        
        model = tf.keras.models.Model(inputs = [inp], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tfa.losses.SigmoidFocalCrossEntropy(alpha = 0.50, gamma = 2.0)],
            metrics = [tf.keras.metrics.AUC(multi_label = True)]
        )

        return model
    
def inference(model_paths):
    
    # Create a numpy array to store predictions
    predictions = np.zeros((NUM_TESTING_IMAGES, N_CLASSES))
    
    print('Extracting test image StudyInstanceUID...')
    # Get the test dataset without tta to extract image names
    test_dataset = get_tta(TEST_FILENAMES, tta = False)
    StudyInstanceUID = test_dataset.map(lambda image, StudyInstanceUID: StudyInstanceUID).unbatch()
    StudyInstanceUID = next(iter(StudyInstanceUID.batch(NUM_TESTING_IMAGES))).numpy().astype('U')
    print('Test image StudyInstanceUID completed...')
    
    for fold, model_path in enumerate(model_paths):
        print('\n')
        print('-'*50)
        print(f'Predicting fold {fold + 1}')
        K.clear_session()
        model = get_model()
        # Load weights of pretrained model
        model.load_weights(model_path)
        
        steps = TTA * ((NUM_TESTING_IMAGES / BATCH_SIZE) + 1)
        # Get the test dataset with tta to extract image
        test_dataset = get_tta(TEST_FILENAMES, tta = True)
        image = test_dataset.map(lambda image, StudyInstanceUID: image)
        probabilities = model.predict(image, steps = steps)[: TTA * NUM_TESTING_IMAGES]
        probabilities = np.mean(probabilities.reshape((NUM_TESTING_IMAGES, TTA, N_CLASSES), order = 'F'), axis = 1)
        predictions += probabilities / len(model_paths)
        
    target_columns = ["ETT - Abnormal", "ETT - Borderline", "ETT - Normal", "NGT - Abnormal", "NGT - Borderline", "NGT - Incompletely Imaged", "NGT - Normal", "CVC - Abnormal", "CVC - Borderline", 
                      "CVC - Normal", "Swan Ganz Catheter Present"]
    predictions_df = pd.DataFrame(predictions, columns = target_columns)
        
    sub = pd.DataFrame({'StudyInstanceUID': StudyInstanceUID})
    sub['StudyInstanceUID'] = [StudyInstanceUID[:-4] for StudyInstanceUID in sub['StudyInstanceUID']]
    sub = pd.concat([sub, predictions_df], axis = 1)
    sub.to_csv('submission.csv', index = False)
        
    return sub
        
# Get pretrained models list for inference
model_paths = glob.glob('../input/ranzcr-effb6-model/*.h5')
sub = inference(model_paths)
sub.head()